In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


## Import dataset

Import the training dataset and save it in dataframe "df0"

In [3]:
df0 = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')

Import test data in df_test dataframe

In [4]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

We'll make changes in dataset. So, it's better to keep original dataset (df0) aside and make changes in copy of original dataset (df)

In [5]:
df = df0.copy()

Read the first five rows of the dataset

In [ ]:
df.head()

Let's dive into each feature.
1. ID- It seems of little to no importance as it is just the id of each potential policy holder.
2. Gender - Categorical Variable. Keeping in ethics of developing machine learning models, it is better to avoid this feature as this field of research should not be depending upon the gender of policy holders. 
3. Age - Age of potential policy holders. This can be of importance as to reach out target audeince.
4. Driving licence - Categorical Variable. Can or cannot be of importance. We'll see.
5. Region Code- Discrete Variable. Can be of importance to reach out our target audeince and to run the campaign and educate peopple in areas with less policy holders.
6. Previously Insured - Categorical Varibale. Of importance.
7. Vehicle Age - Needs to be converted into categorical variable. Of importance.
8. Vehicle Damage - Needs to be converted into categorical variable. Of importance.
9. Annual Premium - Discrete Variable. We'll plot boxplot and deep dive into it. It is of importnace to find out what is the most preferred premium people prefer.
10. Policy Sales Channel - Discrete Variable. Channel through which policy is sold. Can be of importance. Stakeholders can look into less occuring sales channel and can take measures to increase the sales of policy via those channels.
11. Vintage - A vintage car is a prized possession for the owner. It can be of importance.

Now, Familiarize with the datatypes of the features of the dataset

In [ ]:
df.info()

Get the descriptive analysis about the features of the dataset.

In [ ]:
df.describe()

Find out total number of rows and column in the dataset.

In [ ]:
df.shape

In [ ]:
df.head()

Lets' check for total number of duplicated rows in dataset.

In [ ]:
df.duplicated().sum()

LEt's check for missing values in dataset.

In [6]:
missing_values = df.isnull().sum()
missing_values

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

So, we dont have any missing values.

## Exploratory Data Analysis

Let's start EDA.

Let's check the frequency of potential policy buyers on basis of age by plotting histogram.

In [7]:
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
sns.histplot(df['Age'],bins=10)

plt.show()

Let's check the frequency of potential policy buyers on basis of region code by plotting histogram.

In [ ]:
plt.figure(figsize=(5,4))
sns.histplot(df['Region_Code'], kde=False)
plt.title("Distribution of Region Code")
plt.xlabel("Region Code")
plt.ylabel("Frequency")
plt.show()

As each region code specify individual region, hence we are not including "bins" variable in the plot.
We can see that region having region code 28 has most number of policy holders followed by region code 8.
Stakeholders can concentrate to educate other regions regarding insurance policies and it's benefits. 

Now, Let's check the frequency of potential policy buyers on basis of vehicle age by plotting histogram.

In [ ]:
sns.histplot(df['Vehicle_Age'])
plt.show()

Max potential policy holders have vehicle either less than 1year old or less than 2 year old. 
These can be more of our target customers.

Note - We will have to change this variable into categorial variable.

Now,let's do the analysis of the feature annual premium. We'll plot a boxplot for the same. 

In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(df['Annual_Premium'], orient="h")
plt.title("Distribution of Annual Premium")
plt.xlabel("Annual Premium")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df["Annual_Premium"].describe()

We can see that min is 2630 , max is 540165.
In the boxplot, we can see that there are outliers beyyond 60k.
So,
* in first iteration ,we'll see do the analysis with this data .
* And, in second iteration we'll impute the outliers (by changing IQR), and we'll check if it has improved our model results.

Now, as it is a binary classification problem, let's check if there is a class imbalance. 

In [8]:
imbal = df['Response'].value_counts(normalize=True)*100
imbal

Response
0    87.700271
1    12.299729
Name: proportion, dtype: float64

88% of the response varibles is labelled as 0, i.e., negative and 12% of response is 1,i.e. positive

Let's plot pie chart to visualise the same

In [ ]:
response_data = df['Response'].value_counts()
labels = [0,1]
fig , axes = plt.subplots()
axes.pie(response_data,labels)
axes.set_title("Response Variable Imbalance")
plt.show()

## Feature Engineering

### Feature Transformation

Let's change the feature "Vehicle_Damage" from Yes/No category to 0/1 and save it in new feature "Vehicles_Damages"

In [9]:
def vehicle_dam(Vehicle_Damage):
    if Vehicle_Damage == 'No':
        return 0
    else:
        return 1

In [10]:
df['Vehicles_Damages'] = df['Vehicle_Damage'].apply(vehicle_dam)
df['Vehicles_Damages']

0           1
1           1
2           0
3           1
4           0
           ..
11504793    1
11504794    1
11504795    0
11504796    1
11504797    0
Name: Vehicles_Damages, Length: 11504798, dtype: int64

Let's convert the feature "Vehicle_Ages" to dummy varibles (categorical variable).

In [11]:
df['Vehicle_Age']= df['Vehicle_Age'].astype('category')
df = pd.get_dummies(df, columns =['Vehicle_Age'])


In [ ]:
df.head()

In [ ]:
df.shape

Let's alter the test dataset as well to align it with the changes in training data

In [12]:
df_test['Vehicles_Damages'] = df_test['Vehicle_Damage'].apply(vehicle_dam)
df_test['Vehicles_Damages']

0          0
1          1
2          1
3          0
4          0
          ..
7669861    1
7669862    0
7669863    0
7669864    1
7669865    0
Name: Vehicles_Damages, Length: 7669866, dtype: int64

In [13]:
df_test['Vehicle_Age'] = df_test['Vehicle_Age'].astype('category')
df_test = pd.get_dummies(df_test, columns =['Vehicle_Age'])

In [ ]:
df_test.head()

In [ ]:
df_test.shape

I had tried modelling without changing/ transforming Annual Premium column, but the ROC metric was not upto the mark (0.59).
Hence, now I'll try imputing the outliers on Annual Premium column.

In [14]:
def outlier_imputation(column_list, iqr_factor):
    for col in column_list:
        Q3 = df[col].quantile(0.75)
        Q1 = df[col].quantile(0.25)
        IQR = Q3 - Q1
        min = Q1 - (iqr_factor*IQR)
        max = Q3 + (iqr_factor*IQR)
        print("min = ",min)
        print("max = ",max)
        print("25% = ",Q1)
        print("75% = ",Q3)
        
        df.loc[df[col]>max, col] = max
        df.loc[df[col]<min, col] = min

In [15]:
new_df = df.copy()

In [ ]:
new_df['Annual_Premium'].describe()

In [16]:
column_list =["Annual_Premium"]
column_list

['Annual_Premium']

In [17]:
outlier_imputation(column_list,1.5)

min =  4016.0
max =  60712.0
25% =  25277.0
75% =  39451.0


In [ ]:
plt.figure(figsize=(8,7))
sns.boxplot(df['Annual_Premium'], orient="h")
plt.title("Distribution of Annual Premium")
plt.xlabel("Annual Premium")
plt.ylabel("Frequency")
plt.show()

In [ ]:
df= new_df.copy()

## Feature selection

In [ ]:
df.info()

In [18]:
df_X_train = df[['Age','Driving_License','Region_Code','Previously_Insured',
             'Annual_Premium','Policy_Sales_Channel','Vintage',
             'Vehicles_Damages','Vehicle_Age_1-2 Year','Vehicle_Age_< 1 Year',
             'Vehicle_Age_> 2 Years']]

df_y_train = df[['Response']]



## Spliting training and test samples for model training

In [ ]:
print(df_X_train.shape)
print(df_y_train.shape)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X_train, df_y_train, test_size=0.25, random_state=42)

In [ ]:
print(X_train.shape)
print(y_train.shape)

Let's use SMOTE for balamcing unbalanced classes in the dataset.

In [20]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [22]:
imbaln = y_train_resampled['Response'].value_counts(normalize=True)*100
imbaln

Response
0    50.0
1    50.0
Name: proportion, dtype: float64

As, we can see that Annual Premium is in hundreds of thousands, where as the feature Vintage is only in hundreds. This could lead to Annual Premium to add more weightage in the model. 
Hence, it is better to scale our data.

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
X_train_scaled = scaler.fit_transform(X_train_resampled)

As training data has been scaled, hence, we should scale our test data as well with same scaling criteria.

In [26]:
X_test_scaled = scaler.fit_transform(X_test)

In [27]:
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, roc_curve

In [28]:
def model_prediction(model):
    model.fit(X_train_scaled,y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    a = accuracy_score(y_test,y_pred)
    p = precision_score(y_test, y_pred)
    r = roc_auc_score(y_test,y_pred)
    print(f"Accuracy_Score of {model} model on Training Data is:",a)
    print(f"Precision score of {model} model on Training Data is:",p)
    print(f"ROC score of {model} model on Training Data is :",r)
    return model
   
    

In [29]:
print(X_train_scaled.shape)
print(y_train_resampled.shape)

(15134358, 11)
(15134358, 1)


In [30]:
from sklearn.naive_bayes import GaussianNB

model_nb = model_prediction(GaussianNB())
y_pred_nb = model_nb.predict(X_test_scaled)


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy_Score of GaussianNB() model on Training Data is: 0.8606355608094013
Precision score of GaussianNB() model on Training Data is: 0.3000982567890359
ROC score of GaussianNB() model on Training Data is : 0.5337138964846959


In [32]:
from xgboost import XGBClassifier
model_xgb = model_prediction(XGBClassifier())
y_pred_xgb = model_xgb.predict(X_test_scaled)

Accuracy_Score of XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...) model on Training Data is: 0.3525780543773034
Precision score of XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, 

In [ ]:
'''

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
random_forest_clssifier = model_prediction(RandomForestClassifier())

'''

I had ran this cell with all the classifiers and random classifier gave the best results so I ran the cell again the next day with only random classifier (It took 1.5 hours with only one classifier).

Let's predict the test data with our trained model.

In [33]:
X_test_final = df_test[['Age','Driving_License','Region_Code','Previously_Insured',
             'Annual_Premium','Policy_Sales_Channel','Vintage',
             'Vehicles_Damages','Vehicle_Age_1-2 Year','Vehicle_Age_< 1 Year',
             'Vehicle_Age_> 2 Years']]

X_test_final_scaled = scaler.fit_transform(X_test_final)

y_final_pred = model_nb.predict(X_test_final_scaled)


In [ ]:
print(df_test.shape)
print(X_test_final.shape)
id = df_test['id']


In [ ]:
submission = pd.DataFrame({'id':id,'Response':y_final_pred})
submission.to_csv('submission.csv', index=False)
submission

I will try to improve the model further. 

This is the third iteration iteration.

1. 1st iteration - tried with log regression, results were not great.
2. 2nd iteration - tried with random classifier, took 1.5 hours to train.
3. 3rd iteration - imputed the outliers of Annual_Premium column, tried with XGBoost and naive bayes( NB showed better area under ROC, so going with that as of now.) 

If you have any suggestions, feedback, feel free to comment.